<a href="https://colab.research.google.com/github/Samar-Agarwal/Detecting-Depression-through-Tweets/blob/main/wids_nlp_22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM
from keras.utils.np_utils import to_categorical
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data =pd.read_csv("drive/MyDrive/WIDS_NLP_Project/dataset2.csv", on_bad_lines = 'skip')
# Keeping only the neccessary columns
data = data[['Sentiment', 'SentimentText']]

In [ ]:
data['SentimentText'] = data['SentimentText'].apply(lambda x: x.lower()) # lowering all alphabets
data['SentimentText'] = data['SentimentText'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x))  #using re module for stooping

print(data[ data['Sentiment'] == 1].size)
print(data[ data['Sentiment'] == 0].size)

1580354
1576870


In [ ]:
#defining the function to remove punctuation
import string
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree
#storing the puntuation free text
data['SentimentText']= data['SentimentText'].apply(lambda x:remove_punctuation(x))

In [ ]:
#defining function for tokenization
def tokenization(text):
    tokens = re.split(r'\W+',text)
    return tokens
#applying function to the column
data['SentimentText']= data['SentimentText'].apply(lambda x: tokenization(x))

In [ ]:
import nltk
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
#defining the function to remove stopwords from tokenized text
def remove_stopwords(text):
    output= [i for i in text if i not in stopwords]
    return output
#applying the function
data['SentimentText']= data['SentimentText'].apply(lambda x:remove_stopwords(x))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#importing the Stemming function from nltk library
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [ ]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
#defining the object for Lemmatization
wordnet_lemmatizer = WordNetLemmatizer()
#defining the function for lemmatization
def lemmatizer(text):
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm_text
data['SentimentText']=data['SentimentText'].apply(lambda x:lemmatizer(x))

In [ ]:
import gensim

In [ ]:
data.head()


,Sentiment,SentimentText
0,0,"[, sad, apl, friend]"
1,0,"[, missed, new, moon, trailer]"
2,1,"[, omg, already, 730]"
3,0,"[, omgaga, im, sooo, im, gunna, cry, ive, dent..."
4,0,"[, think, mi, bf, cheating, tt]"


In [ ]:
documents = []
for _text in data['SentimentText'].values:
      documents.append(_text) 

In [ ]:
data.head()

,Sentiment,SentimentText
0,0,"[, sad, apl, friend]"
1,0,"[, missed, new, moon, trailer]"
2,1,"[, omg, already, 730]"
3,0,"[, omgaga, im, sooo, im, gunna, cry, ive, dent..."
4,0,"[, think, mi, bf, cheating, tt]"


In [ ]:
print(documents[0:10])

[['', 'sad', 'apl', 'friend'], ['', 'missed', 'new', 'moon', 'trailer'], ['', 'omg', 'already', '730'], ['', 'omgaga', 'im', 'sooo', 'im', 'gunna', 'cry', 'ive', 'dentist', 'since', '11', 'suposed', '2', 'get', 'crown', 'put', '30mins'], ['', 'think', 'mi', 'bf', 'cheating', 'tt'], ['', 'worry', 'much', ''], ['', 'juuuuuuuuuuuuuuuuussssst', 'chillin'], ['', 'sunny', 'work', 'tomorrow', 'tv', 'tonight'], ['', 'handed', 'uniform', 'today', 'miss', 'already'], ['', 'hmmmm', 'wonder', 'number', '']]


In [ ]:
W2V_SIZE = 300
W2V_WINDOW = 7
W2V_EPOCH = 32
W2V_MIN_COUNT = 10
w2v_model = gensim.models.word2vec.Word2Vec(size=W2V_SIZE, 
                                            window=W2V_WINDOW, 
                                            min_count=W2V_MIN_COUNT, 
                                            workers=12)


w2v_model.build_vocab(documents)

In [ ]:
words = w2v_model.wv.vocab.keys()
vocab_size = len(words)
print("Vocab size", vocab_size)

Vocab size 41535


In [ ]:
%%time
w2v_model.train(documents, total_examples=len(documents), epochs=W2V_EPOCH)

CPU times: user 31min 58s, sys: 9.75 s, total: 32min 7s
Wall time: 17min 1s


(338970219, 435690624)

In [ ]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(data['SentimentText'].values)
X = tokenizer.texts_to_sequences(data['SentimentText'].values)
X = pad_sequences(X)

In [ ]:
vocab_size = len(tokenizer.word_index)+1
print('Vocab Size is ',vocab_size)

Vocab Size is  827178


In [ ]:
# texts_new =[]
# for w_list in data['SentimentText'].values:
#   x_new =[]
#   for word in w_list:
#     if (word == ""): continue 
#     if (int(tokenizer.word_index[word])<=2000):
#       x_new.append(word)
#   texts_new.append(x_new)


In [ ]:
# tokenizer_new = Tokenizer(num_words=2000)
# tokenizer_new.fit_on_texts(texts_new)
# X_new = tokenizer_new.texts_to_sequences(texts_new)
# X_new = pad_sequences(X_new)

In [ ]:
# vocab_size_new = len(tokenizer_new.word_index)+1
# print('Vocab Size is ',vocab_size_new)

In [ ]:
# arr = list(tokenizer_new.word_index.keys())
# print(arr[:10])

In [ ]:
tokenizer.word_index['sam']

1406

In [ ]:
print(X)


[[   0    0    0 ...    1   49   62]
 [   0    0    0 ...   28  663 1063]
 [   0    0    0 ...    1  152  119]
 ...
 [   0    0    0 ...   73  528    1]
 [   0    0    0 ...    0 1889    1]
 [   0    0    0 ...    0   48    1]]


In [ ]:
w2v_model.wv['sad'].shape


(300,)

In [ ]:
embed_dim = 300
embedding_matrix = np.zeros(( vocab_size, embed_dim))
for word , i in tokenizer.word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]
print(embedding_matrix.shape)

(827178, 300)


In [ ]:
from keras.models import Sequential
from keras.layers import SpatialDropout1D

lstm_out = 80
model = Sequential()
model.add(Embedding(vocab_size, embed_dim, weights = [ embedding_matrix], input_length = X.shape[1], trainable = False))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 28, 300)           248153400 
                                                                 
 spatial_dropout1d_2 (Spatia  (None, 28, 300)          0         
 lDropout1D)                                                     
                                                                 
 lstm_2 (LSTM)               (None, 80)                121920    
                                                                 
 dense_2 (Dense)             (None, 2)                 162       
                                                                 
Total params: 248,275,482
Trainable params: 122,082
Non-trainable params: 248,153,400
_________________________________________________________________
None


In [ ]:
data.head()
from sklearn.model_selection import train_test_split

In [ ]:
Y = pd.get_dummies(data['Sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.3, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(1105028, 28) (1105028, 2)
(473584, 28) (473584, 2)


In [ ]:
batch_size = 32
epochs = 1
model.fit(X_train, Y_train, batch_size = batch_size, epochs = epochs, verbose=1)

34533/34533 [==============================] - 2493s 72ms/step - loss: 0.5266 - accuracy: 0.7342


In [ ]:
score,acc = model.evaluate(X_test, Y_test, verbose = 1, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

14800/14800 [==============================] - 195s 13ms/step - loss: 0.4902 - accuracy: 0.7633
score: 0.49
acc: 0.76
